# 99 - Modelo Solución Final

Este notebook contiene la solución completa para la competencia de Kaggle.

**Modelo:** Ensemble de Random Forest + LightGBM con weighted average

**Pipeline:**
1. Carga de datos
2. Preprocesamiento (mapeo ordinal, encoding)
3. Entrenamiento de Random Forest
4. Entrenamiento de LightGBM
5. Combinación mediante promedio ponderado
6. Generación de predicciones para test

**Accuracy en Kaggle:** 0.41497

## Instalación e importación de librerías

In [ ]:
# Instalar LightGBM
!pip install lightgbm -q

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

## Carga de datos

In [ ]:
# Cargar datasets
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

## Preprocesamiento

El preprocesamiento incluye:
- Mapeo ordinal para variables con orden natural (estrato, educación, horas de trabajo)
- Encoding binario para variables Si/No
- Label encoding para programa académico
- One-hot encoding para departamento (Random Forest)
- Label encoding para departamento (LightGBM)
- Imputación de valores faltantes con mediana

In [ ]:
def preprocess_base(df, is_train=True):
    """
    Preprocesamiento base común para ambos modelos.
    """
    df_copy = df.copy()
    
    # Guardar ID y target
    ids = df_copy['ID'].copy()
    if is_train:
        target = df_copy['RENDIMIENTO_GLOBAL'].copy()
    
    # Eliminar columnas que no son features
    cols_to_drop = ['ID']
    if is_train:
        cols_to_drop.append('RENDIMIENTO_GLOBAL')
    df_copy = df_copy.drop(cols_to_drop, axis=1)
    
    # Mapeo ordinal para valor de matrícula
    valor_orden = {
        'Menos de 500 mil': 0,
        'Entre 500 mil y menos de 1 millón': 1,
        'Entre 1 millón y menos de 2.5 millones': 2,
        'Entre 2.5 millones y menos de 4 millones': 3,
        'Entre 4 millones y menos de 5.5 millones': 4,
        'Entre 5.5 millones y menos de 7 millones': 5,
        'Más de 7 millones': 6
    }
    df_copy['E_VALORMATRICULAUNIVERSIDAD'] = df_copy['E_VALORMATRICULAUNIVERSIDAD'].map(valor_orden)
    
    # Mapeo ordinal para horas de trabajo semanales
    horas_orden = {
        '0': 0,
        'Menos de 10 horas': 1,
        'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3,
        'Más de 30 horas': 4
    }
    df_copy['E_HORASSEMANATRABAJA'] = df_copy['E_HORASSEMANATRABAJA'].map(horas_orden)
    
    # Mapeo ordinal para estrato
    estrato_map = {
        'Sin Estrato': 0,
        'Estrato 1': 1,
        'Estrato 2': 2,
        'Estrato 3': 3,
        'Estrato 4': 4,
        'Estrato 5': 5,
        'Estrato 6': 6
    }
    df_copy['F_ESTRATOVIVIENDA'] = df_copy['F_ESTRATOVIVIENDA'].map(estrato_map)
    
    # Mapeo ordinal para educación de padres
    educacion_orden = {
        'Ninguno': 0,
        'Primaria incompleta': 1,
        'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3,
        'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5,
        'Técnica o tecnológica completa': 6,
        'Educación profesional incompleta': 7,
        'Educación profesional completa': 8,
        'Postgrado': 9,
        'No sabe': 2
    }
    df_copy['F_EDUCACIONPADRE'] = df_copy['F_EDUCACIONPADRE'].map(educacion_orden)
    df_copy['F_EDUCACIONMADRE'] = df_copy['F_EDUCACIONMADRE'].map(educacion_orden)
    
    # Mapeo binario para variables Si/No
    binary_map = {'Si': 1, 'No': 0, 'S': 1, 'N': 0}
    binary_cols = ['F_TIENEINTERNET', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 
                   'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'E_PAGOMATRICULAPROPIO']
    
    for col in binary_cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].map(binary_map)
    
    df_copy['E_PRIVADO_LIBERTAD'] = df_copy['E_PRIVADO_LIBERTAD'].map({'S': 1, 'N': 0})
    
    # Imputar valores faltantes con mediana
    numeric_cols = df_copy.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df_copy[col].isna().sum() > 0:
            df_copy[col] = df_copy[col].fillna(df_copy[col].median())
    
    return df_copy, ids, target if is_train else None

In [ ]:
def preprocess_for_rf(df_base):
    """
    Preprocesamiento específico para Random Forest.
    Usa one-hot encoding para departamento.
    """
    df_copy = df_base.copy()
    
    # Label encoding para programa académico
    le_programa = LabelEncoder()
    df_copy['E_PRGM_ACADEMICO'] = le_programa.fit_transform(df_copy['E_PRGM_ACADEMICO'].astype(str))
    
    # One-hot encoding para departamento
    depto_dummies = pd.get_dummies(df_copy['E_PRGM_DEPARTAMENTO'], prefix='E_PRGM_DEPARTAMENTO', drop_first=False)
    df_copy = df_copy.join(depto_dummies)
    df_copy = df_copy.drop('E_PRGM_DEPARTAMENTO', axis=1)
    
    return df_copy

In [ ]:
def preprocess_for_lgbm(df_base):
    """
    Preprocesamiento específico para LightGBM.
    Usa label encoding para todas las categorías.
    """
    df_copy = df_base.copy()
    
    # Label encoding para programa y departamento
    le_programa = LabelEncoder()
    le_depto = LabelEncoder()
    
    df_copy['E_PRGM_ACADEMICO'] = le_programa.fit_transform(df_copy['E_PRGM_ACADEMICO'].astype(str))
    df_copy['E_PRGM_DEPARTAMENTO'] = le_depto.fit_transform(df_copy['E_PRGM_DEPARTAMENTO'].astype(str))
    
    return df_copy

In [ ]:
# Aplicar preprocesamiento base
print("Preprocesando train...")
df_train_base, train_ids, target = preprocess_base(df_train, is_train=True)

print("Preprocesando test...")
df_test_base, test_ids, _ = preprocess_base(df_test, is_train=False)

# Crear versiones específicas para cada modelo
print("\nCreando versión para Random Forest...")
df_train_rf = preprocess_for_rf(df_train_base)
df_test_rf = preprocess_for_rf(df_test_base)

print("Creando versión para LightGBM...")
df_train_lgbm = preprocess_for_lgbm(df_train_base)
df_test_lgbm = preprocess_for_lgbm(df_test_base)

print(f"\nTrain RF: {df_train_rf.shape}")
print(f"Train LightGBM: {df_train_lgbm.shape}")

## Preparación de datos para entrenamiento

In [ ]:
# Codificar variable objetivo
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(target)

# Separar features y target para Random Forest
X_rf = df_train_rf.values
X_test_rf = df_test_rf.values

# Separar features y target para LightGBM
X_lgbm = df_train_lgbm.values
X_test_lgbm = df_test_lgbm.values

# Split train/validation con el mismo random_state para ambos modelos
X_train_rf, X_val_rf, y_train, y_val = train_test_split(
    X_rf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

X_train_lgbm, X_val_lgbm, _, _ = train_test_split(
    X_lgbm, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Train: {X_train_rf.shape[0]} muestras")
print(f"Validation: {X_val_rf.shape[0]} muestras")

## Entrenamiento de Random Forest

In [ ]:
print("Entrenando Random Forest...")

# Configuración optimizada para Colab
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='log2',
    random_state=42,
    n_jobs=-1,
    verbose=1
)

rf_model.fit(X_train_rf, y_train)

# Evaluar en validación
y_val_pred_rf = rf_model.predict(X_val_rf)
rf_acc = accuracy_score(y_val, y_val_pred_rf)

print(f"\nAccuracy Random Forest: {rf_acc:.5f}")

## Entrenamiento de LightGBM

In [ ]:
print("Entrenando LightGBM...\n")

# Crear datasets de LightGBM
train_data = lgb.Dataset(X_train_lgbm, label=y_train)
val_data = lgb.Dataset(X_val_lgbm, label=y_val, reference=train_data)

# Configuración optimizada
params_lgbm = {
    'objective': 'multiclass',
    'num_class': 4,
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 64,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 40,
    'verbose': -1,
    'seed': 42
}

lgbm_model = lgb.train(
    params_lgbm,
    train_data,
    num_boost_round=300,
    valid_sets=[val_data],
    valid_names=['valid'],
    callbacks=[lgb.early_stopping(stopping_rounds=30), lgb.log_evaluation(period=50)]
)

# Evaluar en validación
y_val_pred_lgbm_proba = lgbm_model.predict(X_val_lgbm)
y_val_pred_lgbm = np.argmax(y_val_pred_lgbm_proba, axis=1)
lgbm_acc = accuracy_score(y_val, y_val_pred_lgbm)

print(f"\nAccuracy LightGBM: {lgbm_acc:.5f}")

## Ensemble - Weighted Average

Combinamos las predicciones de ambos modelos usando un promedio ponderado.
Los pesos se calculan basándose en el accuracy de cada modelo en validación.

In [ ]:
# Calcular pesos basados en accuracy
total_acc = rf_acc + lgbm_acc
weight_rf = rf_acc / total_acc
weight_lgbm = lgbm_acc / total_acc

print(f"Pesos del ensemble:")
print(f"  Random Forest: {weight_rf:.3f}")
print(f"  LightGBM: {weight_lgbm:.3f}")

# Obtener probabilidades
y_val_proba_rf = rf_model.predict_proba(X_val_rf)
y_val_proba_lgbm = y_val_pred_lgbm_proba

# Promedio ponderado de probabilidades
y_val_proba_ensemble = (weight_rf * y_val_proba_rf) + (weight_lgbm * y_val_proba_lgbm)

# Predicción final
y_val_pred_ensemble = np.argmax(y_val_proba_ensemble, axis=1)

# Evaluar ensemble
ensemble_acc = accuracy_score(y_val, y_val_pred_ensemble)

print(f"\nResultados en validación:")
print(f"  Random Forest:  {rf_acc:.5f}")
print(f"  LightGBM:       {lgbm_acc:.5f}")
print(f"  Ensemble:       {ensemble_acc:.5f}")

## Reporte de clasificación

In [ ]:
# Convertir a labels originales
y_val_labels = le_target.inverse_transform(y_val)
y_val_pred_labels = le_target.inverse_transform(y_val_pred_ensemble)

print("Reporte de clasificación:\n")
print(classification_report(y_val_labels, y_val_pred_labels))

# Matriz de confusión
cm = confusion_matrix(y_val_labels, y_val_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['alto', 'bajo', 'medio-alto', 'medio-bajo'],
            yticklabels=['alto', 'bajo', 'medio-alto', 'medio-bajo'])
plt.title('Matriz de Confusión - Ensemble')
plt.ylabel('Real')
plt.xlabel('Predicho')
plt.tight_layout()
plt.show()

## Entrenamiento de modelos finales

Entrenamos con todos los datos de train para hacer las predicciones finales.

In [ ]:
print("Entrenando modelos finales con todos los datos...\n")

# Random Forest con todos los datos
print("[1/2] Random Forest...")
rf_final = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='log2',
    random_state=42,
    n_jobs=-1,
    verbose=0
)
rf_final.fit(X_rf, y_encoded)

# LightGBM con todos los datos
print("[2/2] LightGBM...")
full_train_data = lgb.Dataset(X_lgbm, label=y_encoded)

lgbm_final = lgb.train(
    params_lgbm,
    full_train_data,
    num_boost_round=lgbm_model.best_iteration,
    callbacks=[lgb.log_evaluation(period=0)]
)

print("\nModelos finales entrenados")

## Generación de predicciones para test

In [ ]:
print("Generando predicciones para test...\n")

# Predicciones de Random Forest
test_proba_rf = rf_final.predict_proba(X_test_rf)

# Predicciones de LightGBM
test_proba_lgbm = lgbm_final.predict(X_test_lgbm)

# Ensemble con weighted average
test_proba_ensemble = (weight_rf * test_proba_rf) + (weight_lgbm * test_proba_lgbm)
test_pred_encoded = np.argmax(test_proba_ensemble, axis=1)

# Convertir a labels originales
test_pred = le_target.inverse_transform(test_pred_encoded)

# Crear archivo de submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': test_pred
})

# Guardar
submission.to_csv('submission.csv', index=False)

print("Archivo submission.csv generado")
print(f"Total de predicciones: {len(submission)}")
print(f"\nDistribución de predicciones:")
print(submission['RENDIMIENTO_GLOBAL'].value_counts())
print(f"\nPrimeras 10 filas:")
print(submission.head(10))

## Resumen

**Modelo utilizado:** Ensemble de Random Forest y LightGBM

**Estrategia de combinación:** Weighted average basado en accuracy de validación

**Resultados:**
- Accuracy en validación mostrado arriba
- Accuracy en Kaggle: 0.41497

**Por qué funciona:**
- Random Forest y LightGBM capturan patrones diferentes en los datos
- Random Forest usa bootstrap sampling y es robusto
- LightGBM usa gradient boosting y es más eficiente
- La combinación ponderada aprovecha las fortalezas de ambos

**Archivo generado:** submission.csv listo para enviar a Kaggle